In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

# Load Mediapipe selfie segmentation model
mp_selfie_segmentation = mp.solutions.selfie_segmentation
selfie_segmentation = mp_selfie_segmentation.SelfieSegmentation(model_selection=1)

# Load ResNet-50 (Assuming it's trained on FashionNet dataset)
fashion_model = tf.keras.applications.ResNet50(weights='imagenet')  # Replace with your trained model

##https://www.youtube.com/watch?v=MCCIQQaRHSg

In [ ]:
def segment_selfie(image):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = selfie_segmentation.process(rgb_image)

    # Create mask
    mask = result.segmentation_mask
    mask = (mask > 0.5).astype(np.uint8) * 255  # Binary mask

    # Apply mask
    segmented = cv2.bitwise_and(image, image, mask=mask)
    return segmented

def preprocess_for_resnet(image):
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)  # ResNet expects batch input
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

def detect_fashion(image):
    processed_img = preprocess_for_resnet(image)
    predictions = fashion_model.predict(processed_img)
    decoded_predictions = tf.keras.applications.resnet50.decode_predictions(predictions, top=3)
    return decoded_predictions

# Capture a selfie
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    segmented_image = segment_selfie(frame)

    # Display
    cv2.imshow('Segmented Selfie', segmented_image)
    


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
def preprocess_for_resnet(image):
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)  # ResNet expects batch input
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

def detect_fashion(image):
    processed_img = preprocess_for_resnet(image)
    predictions = fashion_model.predict(processed_img)
    decoded_predictions = tf.keras.applications.resnet50.decode_predictions(predictions, top=3)
    return decoded_predictions

# Example usage
fashion_predictions = detect_fashion(segmented_image)
print(fashion_predictions)
